In [33]:
import numpy as np
import itertools
import pandas as pd

In [34]:
def generate_beta(n, rho):
    rng = np.random.default_rng(seed=None)
    betas = np.where(rng.random(n) < rho, 0.0, rng.normal(size=n))
    return betas

def generate_r_uniform_hypergraph(n, r, beta):
    edges = []
    degrees = np.zeros(n, dtype=int)

    for comb in itertools.combinations(range(n), r):
        weight_sum = beta[list(comb)].sum()
        p = np.exp(weight_sum) / (1 + np.exp(weight_sum))
        if np.random.rand() < p:
            edges.append(comb)
            for v in comb:
                degrees[v] += 1
    return edges, degrees


def add_gaussian_noise(degrees, epsilon, delta, r):
    c_squared = 2 * np.log(1.25 / delta)
    sigma = np.sqrt(c_squared) * np.sqrt(r) / epsilon
    noise = np.random.normal(0, sigma, size=degrees.shape)
    return degrees + noise

In [35]:
def stable_sigmoid(x):
    out = np.empty_like(x)
    pos = x >= 0
    neg = ~pos
    out[pos] = 1 / (1 + np.exp(-x[pos]))
    exp_x = np.exp(x[neg])
    out[neg] = exp_x / (1 + exp_x)
    return out
    
def neg_log_likelihood(beta, degrees, r):  
    n = len(beta)
    edges = itertools.combinations(range(n), r)

    term1 = 0.0
    for e in edges:
        s = np.sum(beta[list(e)])
        term1 += np.logaddexp(0, s)

    term2 = np.dot(beta, degrees)
    return term1 - term2


def gradient_descent(n, r, noisy_degrees, lr=0.01, max_iter=1000,
                     tol=1e-6, verbose=True):
    beta = np.zeros(n)
    combs = np.array(list(itertools.combinations(range(n), r)))
    success = 0
    for t in range(max_iter):
        beta_sums = np.sum(beta[combs], axis=1)
        sigm = stable_sigmoid(beta_sums)
        grad = np.zeros(n)
        for j in range(r):
            np.add.at(grad, combs[:, j], sigm)

        grad -= noisy_degrees
        grad_norm = np.linalg.norm(grad)

        if grad_norm < tol:
            nll = neg_log_likelihood(beta, noisy_degrees, r)
            print(f"Iter {t}: grad norm = {grad_norm:.4f}, NLL = {nll:.4f}")
            success = 1
            break
        beta -= lr * grad

        if verbose and t % 100 == 0:
            nll = neg_log_likelihood(beta, noisy_degrees, r)
            print(f"Iter {t}: grad norm = {grad_norm:.4f}, NLL = {nll:.4f}")
    return beta, success


def run_simulation(n, r, rho, delta, epsilon):
    beta = generate_beta(n, rho)
    edges, degrees = generate_r_uniform_hypergraph(n, r, beta)

    lr = 0.0001
    max_iter = 3000
    l2_error = []
    linf_error = []

    if epsilon == None:
        beta_hat_0, success = gradient_descent(n, r, degrees, lr, max_iter)
        l2 = np.linalg.norm(beta - beta_hat_0)
        l2_error.append(l2)
        linf = np.abs(beta - beta_hat_0).max()
        linf_error.append(linf)
        print(l2, linf)
        return l2_error, linf_error, success
    
    noisy_degrees = add_gaussian_noise(degrees, epsilon, delta, r)
    beta_hat, success = gradient_descent(n, r, noisy_degrees, lr, max_iter)
    l2 = np.linalg.norm(beta - beta_hat)
    l2_error.append(l2)
    linf = np.abs(beta - beta_hat).max()
    linf_error.append(linf)
    print(l2, linf)
    
    return l2_error, linf_error, success

In [36]:
delta = 1e-5
rho = 0.6
r = 3
e_list = [None, 2, 1.5, 1, 0.5]

for n in [100, 250, 500, 750, 1000]:
    for e in e_list:
        print(f"\n Running for r = {r}, n = {n}, epsilon = {e} -> \n")
        with open(f"l_2 error, n - {n}, r - {r}, e - {e}.csv", 'w') as f1, open(f"l_inf error, n - {n}, r - {r}, e - {e}.csv", 'w') as f2:
            c = 0
            t = 100
            while (c < 20 and t > 0):
                    l2, linf, success = run_simulation(n, r, rho, delta, e)
                    t -= 1
                    print("t = ", 100 - t, "vs c = ", c)
                    if success == 1:
                        c += 1
                        # Write current row to both files
                        f1.write(','.join(map(str, l2)) + '\n')
                        f2.write(','.join(map(str, linf)) + '\n')


 Running for r = 4, n = 100, epsilon = None -> 

Iter 0: grad norm = 189558.8656, NLL = 2425734.4101
Iter 100: grad norm = 0.0884, NLL = 2158277.8875
Iter 189: grad norm = 0.0000, NLL = 2158277.8875
0.06348374869722358 0.019274832185846247
t =  1 vs c =  0
Iter 0: grad norm = 161983.1180, NLL = 2504748.5511
Iter 100: grad norm = 0.0023, NLL = 2327328.8912
Iter 148: grad norm = 0.0000, NLL = 2327328.8912
0.05911184849141468 0.017010331668592116
t =  2 vs c =  1
Iter 0: grad norm = 180957.8244, NLL = 2451839.8661
Iter 100: grad norm = 0.9166, NLL = 2218515.1151
Iter 200: grad norm = 0.0001, NLL = 2218515.1151
Iter 243: grad norm = 0.0000, NLL = 2218515.1151
0.06359659738767097 0.016591696429638092
t =  3 vs c =  2
Iter 0: grad norm = 172743.0653, NLL = 2478597.1153
Iter 100: grad norm = 0.0002, NLL = 2279996.5568
Iter 127: grad norm = 0.0000, NLL = 2279996.5568
0.04999569238388215 0.013952744927173773
t =  4 vs c =  3
Iter 0: grad norm = 160380.6075, NLL = 2516002.1430
Iter 100: grad no

KeyboardInterrupt: 

In [ ]:
m1 = np.loadtxt(f"D:\Projects\Privacy and Beta Hypergraph\simulation\l_2 error, n - 1000, r - 2, e - 1.5.csv", delimiter=',', skiprows = 1)
m2 = np.loadtxt(f"D:\Projects\Privacy and Beta Hypergraph\simulation\l_inf error, n - 1000, r - 2, e - 1.5.csv", delimiter=',', skiprows = 1)
print(m1.mean(), np.std(m1))
print(m2.mean(), np.std(m2))